<a href="https://colab.research.google.com/github/Mishra-0709/Myntra-BlackGriffin/blob/main/mapping_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import stone
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from PIL import Image
from sklearn.cluster import KMeans
import random
import json

In [ ]:
#get some images
with open('trainval.txt','r') as file:
    lines = file.readlines()

    numbers = [int(line.strip()) for line in lines]

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
#our class mappings
color_dict = {
    "#373028": [],
    "#422811": [],
    "#513B2E": [],
    "#6F503C": [],
    "#81654F": [],
    "#9D7A54": [],
    "#BEA07E": [],
    "#E5C8A6": [],
    "#E7C1B8": [],
    "#F3DAD6": [],
    "#FBF2F3": []
}

Using cache found in /Users/kinjalbhattacharyya/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-3 Python-3.11.9 torch-2.2.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
#check functions
def within_threshold(check, threshold):
    avg = sum(check) / 3
    if avg > 230:
        return True
    return all(avg - threshold <= value <= avg + threshold for value in check)


def euclidean_sim(arr,check):
    for ele in arr:
        ele = [e-a for e,a in zip(ele,check)]
        res = np.linalg.norm(ele)/442.4
        if res<=0.2 :
            return False
        if within_threshold(check,5):
            return False
    return True


In [ ]:
#function to extract these things:
def getSkinToneClothColor(image_path):
    res = stone.process(image_path,'color')
    skin_tone = res['faces'][0]['skin_tone']

    image = cv2.imread(image_path)
    results = model(image)

    cropped_image = cv2.imread(image_path)
    labels = results.xyxyn[0][:, -1].numpy()
    coords = results.xyxyn[0][:, :-1].numpy()

    for i in range(0,len(labels)):
        #as 0 represents person in coco dataset
        if labels[i]==0:
            x1,y1,x2,y2,pred = coords[i]
            height,width,_ = image.shape
            x1, y1, x2, y2 = int(x1 * width), int(y1 * height), int(x2 * width), int(y2 * height)
            cropped_image = cropped_image[y1:y2, x1:x2]
            break

    height, width, _ = np.shape(cropped_image)
    data = np.reshape(cropped_image, (height * width, 3))
    data = np.float32(data)

    number_clusters = 7
    kmeans = KMeans(n_clusters=number_clusters)
    kmeans.fit(data)

    centers = kmeans.cluster_centers_
    labels = kmeans.labels_
    rgb_values = []

    for index, row in enumerate(centers):
        # Extract RGB values from the row
        b, g, r = int(row[0]), int(row[1]), int(row[2])
        #put a check of cosine similarity
        if euclidean_sim(rgb_values,[r,g,b]):
            rgb_values.append([r, g, b])

    return skin_tone,rgb_values

In [ ]:
for i in range(1001,2000):
    path = './JPEGImages/'+ str(numbers[i])+'.jpg'
    skintone,values = getSkinToneClothColor(path)
    temp = color_dict[skintone]
    for color in values:
        if euclidean_sim(temp,color):
            temp.append(color)
    color_dict[skintone] = temp

In [ ]:
for color, elements in color_dict.items():
    print(f"{color}: {len(elements)} elements")

#373028: 0 elements
#422811: 4 elements
#513B2E: 9 elements
#6F503C: 11 elements
#81654F: 18 elements
#9D7A54: 15 elements
#BEA07E: 16 elements
#E5C8A6: 13 elements
#E7C1B8: 13 elements
#F3DAD6: 3 elements
#FBF2F3: 4 elements


In [ ]:
#suggest skin tones
test_path = './courtney.jpg'
skin_tone_data = stone.process(test_path,'color')
skin_tone = skin_tone_data['faces'][0]['skin_tone']
print(skin_tone)
print(random.sample(color_dict[skin_tone],3))

#BEA07E
[[170, 97, 11], [161, 140, 115], [241, 50, 62]]


In [ ]:
json_data = json.dumps(color_dict)

# Write JSON data to a file
with open('data.json', 'w') as f:
    f.write(json_data)